In [9]:
%matplotlib inline

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [11]:
%pylab inline
pylab.rcParams['figure.figsize'] = (18, 6)

Populating the interactive namespace from numpy and matplotlib


## Step 1: assembly of the coefficient   

Determine the unknown displacement $w$ in each node of the grid. This grid includes two rows
of fictive nodes just outside the plate (section 1.3). This will first lead to a matrix in terms of
the coordinates $x$ and $y$ (figure 1.3). However, for the implementation of the FDM, this matrix
of unknowns is stores as a vector $\texttt{W}$. Place the unknowns $w_{i,j}$ row per row behind eachother in $\texttt{W}$,
allowing the diffierence equation for each node to be written as a matrix. The coefficients in this
equation, together with the position of the unknowns $w_{i,j}$ on the grid, are the so-called stencil of
the diffierence operator. This stencil can be used to assembly the coefficient matrix $\texttt{K}$. Place the
stencil in the coefficient matrix $\texttt{K}$ for each node in the plate and on the edge of the plate. There
will be less equations than unknowns, as explained in section 1.3. Therefore empty rows have
to be reserved in the coefficient matrix $\texttt{K}$ and in the vector of unknowns $\texttt{W}$. The introduction of
the boundary conditions will yield the necessary extra equations.

#### tips: 

Python is row-major order (like C), while Matlab is column-major order(like Fortran)

In [ ]:
#Step 1: filling in the stencil in the coefficient matrix K

# initialize essential parameters
# ... material properties
# ... plate dimensions
# ... amount of gridpoints
N = 11          # amount of gridpoints INSIDE the plate in 1 direction
# ... load

# determine auxiliary variables
n = ???

# determine coefficients in the stencil

# initialize the coefficient matrix K and the right hand side F
K = np.zeros((n**2, n**2))
F = np.zeros((n**2, 1))

# the degrees of freedom are stored in the column vector W, for which:
#    1)  K * W = F
#    2)  W(k) = w(i,j) with w(i,j) the displacement of gridpoint (i,j)
# fill in the stencil in the coefficient matrix K and the right hand side F. 
for i in ???:    # ............. loop over the rows INSIDE the plate, 0, 1, N+2, N+3 are ghost cells!!!
    for j in ???:    # ............. loop over the gridpoints inside each row INSIDE the plate
        
        k = ???    # rownumber of the corresponding degree of freedom in W


Kst = K      

# visualisation of the coefficient matrix
plt.figure(1)
plt.xlabel(' percentage non-zero = ' '%.4f' % (np.count_nonzero(K)/(n**2))) # add x-label
plt.title('coefficient matrix K')
stencil_patch = mpatches.Patch(color='blue', label='stencil')
plt.legend(handles=[stencil_patch])
plt.spy(Kst, color='blue', markersize=2)
plt.show()

## Step 2: Introduction of the symmetry boundary conditions

At a symmetry edge, two rows of nodes are located outside the plate. Find for each of these
fictive nodes an algebraic equation, replacing an empty row in the coficient matric $\texttt{K}$. The
algebraic equations can be found from the mathematical description of the symmetry boundary
condition (equation (1.7)).

In [ ]:
# Step 2: entering the symmetry boundary conditions

# initialize the coefficient matrix Ks ( of symmetry boundary condition)
Ks = np.zeros((n**2, n**2))
# y-symmetry for boundary 1

# x-symmetry for boundary 4

# adapting the coefficient matrix K
K = K + Ks

# visualisation of the coefficient matrix
plt.figure(1)
plt.xlabel(' percentage non-zero = ' '%.4f' % (np.count_nonzero(K)/(n**2))) # add x-label
plt.title('coefficient matrix K')
symmetry_patch = mpatches.Patch(color='red', label='symmetry BC')
plt.legend(handles=[stencil_patch, symmetry_patch])
plt.spy(Kst, color='b', markersize=2)
plt.spy(Ks, color='r', markersize=2)
plt.show()

## Step 3: Introduction of the boundary conditions for the clamped and simply sup-ported edge

As for a symmetry edge, two rows of nodes are located outside the plate at the clamped and the simply supported edge. However, for these edges the values at the nodes on the edge are already known. In the partitioning step (step 5), it will become clear that the stencils of these nodes are not used. Therefore the second row of points outside the plate are not needed. Find only for the first row of fictive points outside the plate a system of algebraic equations, starting from the mathematical description (equations (1.5) and (1.6)).

In [ ]:
# Step 3: entering the simply supported and clamped boundary conditions

# initialize the coefficient matrix Ki (clamped)
#                coefficient matrix Ko (support)
Ki = np.zeros((n**2, n**2))  # Ki is a sparse matrix
Ko = np.zeros((n**2, n**2))  # Ko is a sparse matrix

# Boundary condition of clamped boundary (2)
# (contribution of boundary point itself will be accounted for by matrix partitioning)

# Boundary condition of supported boundary (3)
# (contribution of boundary point itself will be accounted for by matrix partitioning)

# adapting the coefficient matrix
K = K + Ki + Ko

# visualization of the coefficient matrix
plt.figure(1)
plt.xlabel(' percentage non-zero = ' '%.4f' % (np.count_nonzero(K)/(n**2))) # add x-label
plt.title('coefficient matrix K')
clamped_patch = mpatches.Patch(color='m', label='clamped BC')
supported_patch = mpatches.Patch(color='g', label='supported BC')
plt.legend(handles=[stencil_patch, symmetry_patch, clamped_patch, supported_patch])
plt.spy(Kst, color='b', markersize=2)
plt.spy(Ks, color='r', markersize=2)
plt.spy(Ki, color='m', markersize=2) 
plt.spy(Ko, color='g', markersize=2)
plt.show()        

## Step 4: Introduction of the contributions of the 4 corner nodes just outside the plate

The 4 corner nodes, just outside the plate, are fictive points. Find an algebraic equation for each of these corner nodes. The edge whereto these nodes belong can be chosen freely.

In [ ]:
# Step 4: entering the contribution of the 4 cornerpoints just outside the plate

# initialize the coefficient matrix K4 (4 cornerpoints)
K4 = np.zeros((n**2, n**2))  # K4 is a sparse matrix

# cornerpoint 1 (bottom left) -> boundary 1: y-symmetry

# cornerpoint 2 (bottom right) -> boundary 2: clamped

# cornerpoint 3 (top right) -> boundary 3: support

# cornerpoint 4 (top left) -> boundary 4: x-symmetry

# adapt the coefficient matrix
K = K + K4

# visualization of the coefficient matrix
plt.figure(1)
#plt.xlabel(' percentage non-zero = ' + str(K.nnz/(n**2))) # add x-label
plt.xlabel(' percentage non-zero = ' '%.4f' % (np.count_nonzero(K)/(n**2))) # 给 x 轴添加标签
#plt.ylabel('y') # add y-label
plt.title('coefficient matrix K')
coners_patch = mpatches.Patch(color='k', label='4 cornerpoints ')
plt.legend(handles=[stencil_patch, symmetry_patch, clamped_patch, supported_patch, coners_patch])
plt.spy(Kst, color='b', markersize=2)
plt.spy(Ks, color='r', markersize=2)
plt.spy(Ki, color='m', markersize=2) 
plt.spy(Ko, color='g', markersize=2)
plt.spy(K4, color='k', markersize=2)
plt.show()      

## Step 5: Partitioning de coefficient matrix $\texttt{K}$

The partitioning of the coefficient matrix $\texttt{K}$ accounts for the prescribed displacements on the
simply supported and clamped edges, the super
uous second row of dummy points at the simply
supported and clamped edges and the 12 super
uous nodes of the grid. The rows and columns
corresponding to these nodes in the coefficient matrix $\texttt{K}$ are super
uous and have to be removed
in order to obtain a nonsingular coefficient matrix. Within the Matlab environment, it is not
necessary to physically remove these rows and columns. For now, it is sufficient to find the row
numbers of these nodes in the vector $\texttt{W}$ and to store them in the vector $\texttt{k-weg}$.

In [ ]:
# Step 5: partitioning the coefficient matrix 
'''
 takes into account:
    1) Dirichlet conditions on the boundary
    2) 4*3 superfluous cornerpoints
    3) superfluous gridpoints on 2nd row outside supported and clamped boundary
'''

# determine the indices, that stand for the row numbers in W ...

# ... of the gridpoints ON the clamped and simply supported boundaries
k1 = ???

# ... of the 4*3 superfluous boundaries
k2 = ???

# ... of the superfluous gridpoints on the 2nd row outside the simple support and clamped boundaries
k3 = ???
    
# concatenate all degrees of freedom that can be removed
k_remove = set()    
k_remove = k_remove.union(k1, k2, k3)

# determine which dofs have to stay 
k_total = set(range(n**2))
k_save = list(k_total.difference(k_remove))
k_remove = list(k_remove)

# visualization of the matrix partitioning
plt.figure(1)
plt.xlabel(' percentage non-zero = ' '%.4f' % (np.count_nonzero(K)/(n**2))) # add x-label
plt.title('coefficient matrix K')
plt.legend(handles=[stencil_patch, symmetry_patch, clamped_patch, supported_patch, coners_patch])
plt.spy(Kst, color='b', markersize=2)
plt.spy(Ks, color='r', markersize=2)
plt.spy(Ki, color='m', markersize=2) 
plt.spy(Ko, color='g', markersize=2)
plt.spy(K4, color='k', markersize=2)
for k in range(1, len(k_remove)+1):
    plt.plot([1,n**2+1],[k_remove[k-1],k_remove[k-1]], color='k', linewidth=0.6)
    plt.plot([k_remove[k-1],k_remove[k-1]],[1,n**2], color='k', linewidth=0.6)
plt.show()

In [31]:
# Import module in postprocess.py for plotting
import postprocess

## Step 6: Solving the system of algebraic equations

Solve the partitioned system of algebraic equations. Store the solution vector $\texttt{W}$ in an N x N
matrix $\texttt{w}$ for visualization and interpretation. For more information on how to fill this matrix $\texttt{w}$,
use the 'help' function of the Matlab command surf. Use the program $\texttt{ref_opl}$ to visualize
the results and to compare them with the results of a more detailed finite element simulation.
Draw conclusions on the accuracy of the obtained solution.

#### tips:
the linear system can be solved for specific list of index

In [25]:
# Step 6: solve the partitioned linear system K'*W'=F' and visualize

# initialize the total solution vector W
W = np.zeros((n**2, 1))

# solve the partitioned system, k_save indexed only


# convert the solution vector W to a 2D matrix to be visualized with surf
w = np.zeros((N, N))
for i ???:
    for j ???:

x = np.zeros(N)
for i in range(N):
    x[i] = i*dx

postprocess.surf_plot(x, x, w)

## Step 7: Postprocessing - calculation of the stresses

Calculate the stresses at the lower side of the plate $z=-t/2$ according to the dfinitions of
equation (1.8). The program $\texttt{ref_opl}$ calculates the Von Mises stress. The stresses consist
of combinations of derivatives of the calculated deformations. Find the stencil for each stress
component and use it to assemble a new coficient matrix $\texttt{KS}$. Calculate the stress components
with a simple matrix-vector multiplication and save the results in matrix form (as in step 6)
for the visualization and interpretation using the program $\texttt{ref_opl}$. Draw conclusions on the
accuracy of the obtained solution.

In [ ]:
# Step 7: postprocessing - stress calculations for gridpoints inside the plate
    
# enter the location at which stress is calculated
z = -.5 * t    # bottom side
 
# determine auxiliary variables 
S_f = ???    # stress factor in N/m

# determine coefficients in stencil for stress calculation
 
# initialize matrices KSxx, KSyy, KSxy
KSxx = np.zeros((n**2, n**2))
KSyy = np.zeros((n**2, n**2))
KSxy = np.zeros((n**2, n**2))
 
#  enter the stencil in the coefficient matrices
for i in ???:
    for j in ???: 
        
#apply stress factor
KSxx = S_f * KSxx
KSyy = S_f * KSyy
KSxy = S_f * KSxy
 
# visualization of the coefficient matrices
plt.figure(2)
plt.suptitle('coefficient matrix')
ax1 = plt.subplot(331)
plt.spy(KSxx, color='blue', markersize=2)
#plt.setp(ax1.get_xticklabels(), fontsize=6)
plt.xlabel(' pct. non-zero = ' '%.4f' % (np.count_nonzero(KSxx)/(n**2))) # add x-label
plt.title('KSxx', y=1.16)
#stencil_patch = mpatches.Patch(color='blue', label='stencil')
#plt.legend(handles=[stencil_patch])

ax2 = plt.subplot(332)
plt.spy(KSyy, color='blue', markersize=2)
#plt.setp(ax2.get_xticklabels(), fontsize=6)
plt.xlabel(' pct. non-zero = ' '%.4f' % (np.count_nonzero(KSyy)/(n**2))) # add x-label
plt.title('KSyy', y=1.16)

ax3 = plt.subplot(333)
plt.spy(KSxy, color='blue', markersize=2)
#plt.setp(ax3.get_xticklabels(), fontsize=6)
plt.xlabel(' pct. non-zero = ' '%.4f' % (np.count_nonzero(KSxy)/(n**2))) # add x-label
plt.title('KSxy', y=1.16)
plt.show()

In [ ]:
# calculate the stresses
Sxx = ???
Syy = ???
Sxy = ???
 
# save solution vectors in a matrix, to be visualized with surf
s_xx = np.zeros((N,N))
s_yy = np.zeros((N,N))
s_xy = np.zeros((N,N))
for j in ???:
    for i in ???:
        k = i + n*j

# plot the results
postprocess.surf_plot(x, x, w, s_xx, s_yy, s_xy)

## Step 8: Postprocessing - check the boundary conditions

Check if all boundary conditions are satisfied.

In [ ]:
# Step 8: postprocessing - controlling the boundary conditions
# y symmetry for boundary 1 
for i in ???:

# clamped boundary for boundary 2
for j in ???:
    
# simply supported boundary for boundary 3
for i in ???:
    
# x symmetry for boundary 4
for j in ???: